# Problema de atribuição de recursos

## Descrição do problema

- Considere três ofertas de trabalho: Cientista de Dados, Desenvolvedor e Arquiteto.

- Considere três pessoas(recursos): Carlos, Jonas e Monica.

- Considere uma medida(\%) de competência pessoa/trabalho: percentual de competência.

- Assuma que somente uma pessoa pode ser atribuida a um trabalho e no máximo um trabalho pode ser atribuido a uma pessoa.

- Determine uma atribuição que assegure que cada trabalho é executado e cada pessoa é atribuido a no máximo um trabalho onde o valor percentual é maximizado.


## Dados

- A medida de competência/habilidade para a execução de cada trabalho por cada pessoa é ilustrado na tabela a seguir através de um valor percentual.

|            | cientista | desenvolvedor | arquiteto |
|------------|-----------|---------------|-----------|
| Carlos     | 53\%      | 27 \%         | 13 \%     |
| Jonas      | 80 \%     | 47 \%         | 67 \%     |
| Monica     | 53 \%     | 73 \%         | 47 \%     |


## Modelagem

- Variáveis de decisão: 

    $
    x_{i,j} =
    \begin{cases} 
    1, & \text{se a pessoa {\it i} é atribuida ao trabalho {\it j}} \\
    0, & \text{caso contrário}
    \end{cases}
    $ 

    para $i=1,2,3$ e $j=1,2,3$.


- Restrições:

    - Restrição de trabalho: Para cada trabalho $j=1,2,3$, exatamente uma pessoa $i$ deve ser atribuida sobre $i=1,2,3$.
        - Cientista de Dados, j=1:  $x_{1,1} + x_{2,1} + x_{3,1} = 1$
        - Desenvolvedor, j=2: $x_{1,2} + x_{2,2} + x_{3,2} = 1$
        - Arquiteto, j=3: $x_{1,3} + x_{2,3} + x_{3,3} = 1$

    - Restrição de recursos: Para cada recurso $i=1,2,3$, no máximo um trabalho $j$ pode ser atribuido sobre $j=1,2,3$.
        - Carlos, i=1: $x_{1,1} + x_{1,2} + x_{1,3}  \leq 1$
        - Jonas, i=2: $x_{2,1} + x_{2,2} + x_{2,3}  \leq 1$
        - Monica, i=3: $x_{3,1} + x_{3,2} + x_{3,3}  \leq 1$

- Função objetivo: maximize o percentual total máximo associado a atribuições enquanto as restrições de trabalhos e recursos são satisfeitas.
$
\begin{align}
\max \ &  (53 x_{1,1} + 80x_{2,1} + 53x_{3,1}) + (27x_{1,2} + 47x_{2,2} + 73x_{3,2}) + (13x_{1,3} + 67x_{2,3} + 47x_{3,3})
\end{align}
$


## Formulação do problema

$
\begin{align} \notag
\max \ &  (53 x_{1,1} + 80x_{2,1} + 53x_{3,1}) + (27x_{1,2} + 47x_{2,2} + 73x_{3,2}) + (13x_{1,3} + 67x_{2,3} + 47x_{3,3}) \\ \notag
& x_{1,1} + x_{2,1} + x_{3,1} = 1 \\ \notag
& x_{1,2} + x_{2,2} + x_{3,2} = 1  \\ \notag
& x_{1,3} + x_{2,3} + x_{3,3} = 1 \\ \notag
& x_{1,1} + x_{1,2} + x_{1,3}  \leq 1 \\ \notag
& x_{2,1} + x_{2,2} + x_{2,3}  \leq 1 \\ \notag
& x_{3,1} + x_{3,2} + x_{3,3}  \leq 1 \\ \notag
& x_{i,j} \in \{ 0, 1 \} \; \forall \; i=1,2,3, \; j=1,2,3.\notag
\end{align}
$

## Formulação do problema compacta
$
\begin{align} \notag
\max \ & \sum_{j \in J} \sum_{i \in I} perc_{i,j}x_{i,j} \\ \notag
& \sum_{i \in I} x_{i,j} = 1 \; \; \; \forall \; j \in J \\ \notag
& \sum_{j \in J} x_{i,j} \leq 1 \; \; \; \forall \; i \in I \\ \notag
& x_{i,j} \in \{ 0, 1 \} \; \; \; \forall \; i \in I, \; \; \forall \; j \in J. \notag
\end{align}
$


In [ ]:
# importando as bibliotecas
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# conjuntos: recursos e trabalhos
I = ['Carlos', 'Jonas', 'Monica']
J = ['Cientista', 'Desenvolvedor', 'Arquiteto']

In [ ]:
# matriz percentual
key, perc = gp.multidict({
    ('Carlos', 'Cientista'): 53,
    ('Carlos', 'Desenvolvedor'): 27,
    ('Carlos', 'Arquiteto'): 13,
    ('Jonas', 'Cientista'): 80,
    ('Jonas', 'Desenvolvedor'): 47,
    ('Jonas', 'Arquiteto'): 67,
    ('Monica', 'Cientista'): 53,
    ('Monica', 'Desenvolvedor'): 73,
    ('Monica', 'Arquiteto'): 47
})

In [ ]:
# definindo o modelo
model = gp.Model("atribuicao")

In [ ]:
# adicionando variaveis
x = model.addVars(key, name='x')

In [ ]:
type(key)

In [ ]:
key

In [ ]:
type(perc)

In [ ]:
type(x)

In [ ]:
x

In [ ]:
model.update()

In [ ]:
x

In [ ]:
# adicionando restricoes de trabalho
trabalho = model.addConstrs((x.sum('*',j) == 1 for j in J), 'trabalho')

In [ ]:
# adicionando restricoes de recursos(pessoas)
pessoa = model.addConstrs((x.sum(i,'*') <= 1 for i in I), 'pessoa')

In [ ]:
# definindo a funcao objetivo
model.setObjective(x.prod(perc),GRB.MAXIMIZE)

In [ ]:
model.write('atribuicao.mps')

In [ ]:
model.optimize()

In [ ]:
#imprimindo solucao
for var in model.getVars():
    if abs(var.x) > 1e-6:
        print(f"{var.varName} =  {var.x}")

In [ ]:
# imprimindo valor objetivo
print(f"Percentual total = {model.objVal}")

## Um novo cenário

- Considere um novo cenário onde uma nova pessoa, Ana, com percentual de competência igual a 100\% para todos os trabalhos é inserida no modelo.

- Restrições

    - Restrição de trabalho: Para cada trabalho $j=1,2,3$, exatamente uma pessoa $i$ deve ser atribuida sobre $i=1,2,3,4$.
        - Cientista, i=1: $$x_{1,1} + x_{2,1} + x_{3,1} + x_{4,1} = 1$$
        - Desenvolvedor, i=2: $$x_{1,2} + x_{2,2} + x_{3,2} + x_{4,2} = 1$$
        - Arquiteto, i=3: $$x_{1,3} + x_{2,3} + x_{3,3} + x_{4,3}= 1$$

    - Restrição de recursos: Para o recurso sobre $i=4$ no máximo um trabalho $j$  deve ser atribuido sobre $j=1,2,3$.
        - Ana, j=4: $$x_{4,1} + x_{4,2} + x_{4,3}  \leq 1$$


- Função objetivo: maximizar o percentual total associado as atribuições enquanto as restrições de trabalho e recursos são satisfeitas.
$
\begin{align} \notag
\max \ &  (53x_{1,1} + 80x_{2,1} + 53x_{3,1} + 100x_{4,1}) + \\ \notag
& (27x_{1,2} + 47x_{2,2} + 73x_{3,2} + 100x_{4,2}) + \\ \notag
& (13x_{1,3} + 67x_{2,3} + 47x_{3,3} + 100x_{4,3})
\end{align}
$

- Formulação do problema.

$
\begin{align} \notag
\max \ &  (53x_{1,1} + 80x_{2,1} + 53x_{3,1} + 100x_{4,1}) + \\ \notag
& (27x_{1,2} + 47x_{2,2} + 73x_{3,2} + 100x_{4,2}) + \\ \notag
& (13x_{1,3} + 67x_{2,3} + 47x_{3,3} + 100x_{4,3}) \\ \notag
& x_{1,1} + x_{2,1} + x_{3,1} + x_{4,1} = 1 \\ \notag
& x_{1,2} + x_{2,2} + x_{3,2} + x_{4,2} = 1  \\ \notag
& x_{1,3} + x_{2,3} + x_{3,3} + x_{4,3} = 1 \\ \notag
& x_{1,1} + x_{1,2} + x_{1,3}  \leq 1 \\ \notag
& x_{2,1} + x_{2,2} + x_{2,3}  \leq 1 \\ \notag
& x_{3,1} + x_{3,2} + x_{3,3}  \leq 1 \\ \notag
& x_{4,1} + x_{4,2} + x_{4,3}  \leq 1 \\ \notag
& x_{i,j} \in \{ 0, 1 \} \; \forall \; i=1,2,3,4, \; j=1,2,3. 
\end{align}
$

In [ ]:
# alterando o cenario. Ana passa a fazer parte da selecao
nperc = {('Ana','Cientista'):100, ('Ana','Desenvolvedor'):100, ('Ana','Arquiteto'):100}

In [ ]:
for key, val in nperc.items():
    i, j = key
    x[key] = model.addVar(obj=val,
                    name='x[{0},{1}]'.format(i,j),
                    column=gp.Column([1],[model.getConstrByName('trabalho[{0}]'.format(j))])
                   )

In [ ]:
model.addConstr(x.sum('Ana','*') <= 1, name='pessoa[Ana]')

In [ ]:
# resolver novamente
model.optimize()

In [ ]:
# imprimindo solucao
for var in model.getVars():
    if abs(var.x) > 1e-6:
        print(f"{var.varName} = {var.x}")

In [ ]:
# imprimindo o valor otimo
print(f"Novo percentual = {model.objVal}")

In [ ]:
# clear model
model.dispose()